Methods and Experiments
=============

1. Experimental Setup
----------------------
Enumerate all trajectories for each user given the trajectory length (e.g. 3, 4, 5) and the (start, end) POIs.

For each trajectory, compute a score based on the features below:
 * User Interest (time)
 * User interest (frequency)
 * POI Popularity
 * Travelling Cost
 * Trajectory probability based on the transition probabilities between different POI categories and the following rules for choosing a specific POI within certain category:
  * The Nearest Neighbor of the current POI
  * The most Popular POI
  * A random POI choosing with probability proportional to the reciprocal of its distance to current POI
  * A random POI choosing with probability proportional to its popularity

Plot the scores of generated and actual trajectories for each (user, trajectoryLength, startPOI, endPOI) tuple with some degree of transparency (alpha).

Recommend trajectory with the highest score and measure the performance of recommendation using recall, precision and F1-score.

Optimise parameters in the score function by learning, in this specific case, the cost function could be based on recall, precision or F1-score, we can also control the estimation of transition matrix.